In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as data_set
from torchsummary import summary
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np


print(torch.__version__, torchvision.__version__)
print(torch.cuda.get_device_name(0))

1.8.1+cu102 0.9.1+cu102
TITAN RTX


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
max_epochs = 30
print(device)

cuda


In [3]:
vgg16 = torchvision.models.vgg16(pretrained=True)#.to(device)
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [58]:
class LayerResult:
    def __init__(self, layers, layer_index):
        self.hook = layers[layer_index].register_forward_hook(self.hook_fn)
    def hook_fn(self,module, input,output):
        self.features = output.cpu().data.numpy()
    def unregister_forward_hook(self):
        self.hook.remove()

In [61]:
result = LayerResult(vgg16.features, 15)
result.hook

In [4]:
layers = vgg16.children()
ct = 0
layer_level = 1
back_layer = []
for layer in layers:
    print(ct,layer)
    if layer_level is 1:
        for layer_1 in layer:    
            print("layer 1",layer_1)
            if ct >=24 :
                back_layer.append(layer_1)
            ct +=1
    elif layer_level is 2:
        print("layer 2",layer)  
        back_layer.append(layer)
        ct +=1
    elif layer_level is 3:
        for layer_1 in layer: 
            print("layer 3",layer_1) 
            back_layer.append(layer_1)
            ct+=1
    layer_level +=1
    

0 Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_

In [5]:
print(back_layer)
back_layer = nn.Sequential(*back_layer)
'''
if torch.cuda.device_count() > 1 :
    back_layer = nn.DataParallel(back_layer).to(device)
'''

[Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), AdaptiveAvgPool2d(output_size=(7, 7)), Linear(in_features=25088, out_features=4096, bias=True), ReLU(inplace=True), Dropout(p=0.5, inplace=False), Linear(in_features=4096, out_features=4096, bias=True), ReLU(inplace=True), Dropout(p=0.5, inplace=False), Linear(in_features=4096, out_features=1000, bias=True)]


'\nif torch.cuda.device_count() > 1 :\n    back_layer = nn.DataParallel(back_layer).to(device)\n'

In [6]:
print(back_layer)

Sequential(
  (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU(inplace=True)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): AdaptiveAvgPool2d(output_size=(7, 7))
  (8): Linear(in_features=25088, out_features=4096, bias=True)
  (9): ReLU(inplace=True)
  (10): Dropout(p=0.5, inplace=False)
  (11): Linear(in_features=4096, out_features=4096, bias=True)
  (12): ReLU(inplace=True)
  (13): Dropout(p=0.5, inplace=False)
  (14): Linear(in_features=4096, out_features=1000, bias=True)
)


In [7]:

error_feature_path = "/media/2/Network/extracted_feature/whole_not_shuffle_to_15/seq_16_pkt_error"
feature_list =  os.listdir(error_feature_path)
feature_list = sorted(feature_list)
len(feature_list)
# load feature from disk
# classify feature
test_feature = []
test_label = []
train_feature = []
train_label =[]
val_feature = []
val_label =[]
for data in feature_list:
    #print(feature4_path+"/"+data)
    if 'train' in data:
        if 'feature' in data:
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            tmp = torch.from_numpy( np.transpose(tmp, (0,3,1,2)) )
            #print(tmp.shape)
            #tmp = np.transpose(tmp,(0,2,3,1))
            train_feature.append(tmp)
        else :
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            tmp = torch.from_numpy( tmp )
            train_label.append(tmp)
       #print("train in",data)
    elif 'test' in data:
        #print("test in",data)
        if 'feature' in data:
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            tmp = torch.from_numpy( np.transpose(tmp, (0,3,1,2)) )
            test_feature.append(tmp)
        else:
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            tmp = torch.from_numpy( tmp )
            test_label.append(tmp)
    elif 'validation' in data:
        #print("val in",data)
        if 'feature' in data:
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            tmp = torch.from_numpy( np.transpose(tmp, (0,3,1,2)) )
            val_feature.append(tmp)
        else:
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            tmp = torch.from_numpy( tmp )
            val_label.append(tmp)


In [48]:
import torchvision.transforms as T
from torch.utils.data import DataLoader
class custom_Dataset(torch.utils.data.Dataset):
    'Characterize a dataset for PyTorch'
    def __init__(self, imgs,labels):
        self.imgs = imgs
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self,index):
        label = self.labels[index]
        idx1 = int(index / 16384)
        idx2 = index % 16384
        image = self.imgs[idx1][idx2]
        X = self.transform(image)
        Y = self.transform(label)
        return X,Y
    transform = T.Compose([
        T.ToPILImage(),
        T.ToTensor()
    ])
batch_size = 64
train_dataset = DataLoader(custom_Dataset(imgs=train_feature,labels=train_label), batch_size, shuffle=False, num_workers=1)
val_dataset = DataLoader(custom_Dataset(imgs=val_feature,labels=val_label), batch_size, shuffle=False, num_workers=1)
test_dataset = DataLoader(custom_Dataset(imgs=test_feature,labels=test_label), batch_size, shuffle=False, num_workers=1)

In [49]:
for i,data in enumerate(train_dataset):
    print(i,data)
    break

ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/hwbae0326/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/hwbae0326/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/hwbae0326/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-48-9ebbb50ca152>", line 15, in __getitem__
    X = self.transform(image)
  File "/home/hwbae0326/anaconda3/lib/python3.7/site-packages/torchvision/transforms/transforms.py", line 60, in __call__
    img = t(img)
  File "/home/hwbae0326/anaconda3/lib/python3.7/site-packages/torchvision/transforms/transforms.py", line 179, in __call__
    return F.to

In [29]:

criterion = torch.nn.CrossEntropyLoss(size_average = None)#.to(device)
optimizer = torch.optim.SGD(vgg16.parameters(),lr=0.0005)
lr_sche = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [31]:
# evaluation pahse
val_losses=[]
back_layer.eval()
total = 0
correct =0
with torch.no_grad():
    for i, data in enumerate(train_dataset):
        inputs,labels = data
        print(inputs.shape)
        #inputs,labels = inputs.to(device), labels.to(device)
        
        y_hat = back_layer(inputs)
        _, predicted = torch.max(y_hat.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        val_loss = criterion(y_hat, labels)
        val_losses.append(val_loss.item())
        if(i%100 == 99):
            print(".",end="")
    print("")
print("acc of back_layer on imagenet : %d %%" %(100*correct/total))    
'''
back_layer summary 시 (512,14,14)로 input shape 넣으면 되지 않고
npy에 저장한 shape은 (feature개수, w,h, ch)인데, 여기서는 (ch,w,h) 를 요구한다. -> transpose로 해결
210614_1733 - npy가 전부가 들어가니까 800기가가 램에 할당하려고해서 batch를 만들기위해 DataLoader를 사용했다 
하지만 아직 NotImmplemented Error 뜨는중

'''

NotImplementedError: Caught NotImplementedError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/hwbae0326/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/hwbae0326/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/hwbae0326/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/hwbae0326/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataset.py", line 33, in __getitem__
    raise NotImplementedError
NotImplementedError


In [32]:
for i,data in enumerate(train_dataset):
    print(data)
    break

NotImplementedError: Caught NotImplementedError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/hwbae0326/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/hwbae0326/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/hwbae0326/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/hwbae0326/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataset.py", line 33, in __getitem__
    raise NotImplementedError
NotImplementedError
